In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType

In [2]:
project_number = 294601891609
project_id = "crypto-busting-374123"
location = "europe-central2"
subscription_id = "bda-coinbase-topic-sub"
topic_id = "bda-coinbase-topic"
timeout = 5.0

In [3]:
# spark = SparkSession.builder.appName("Read Pub/Sub Stream").master("yarn").getOrCreate()

In [4]:
# spark.sql('add jar file:////home/bda_reddit/dependancies/spark-streaming-pubsub_2.11.jar')

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used
23/01/14 01:18:13 WARN org.apache.hadoop.hive.ql.session.SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


DataFrame[result: int]

In [5]:
# sdf = (
#     spark.readStream.format("pubsub")
#     .option(
#         "pubsub.subscription",
#         f"projects/{project_number}/locations/{location}/subscriptions/{subscription_id}",
#     )
#     .load()
# )

In [6]:
from concurrent.futures import TimeoutError
from google.cloud import pubsub_v1

In [19]:
subscriber = pubsub_v1.SubscriberClient()

In [20]:
subscription_path = subscriber.subscription_path(project_id, subscription_id)

In [21]:
def callback(message: pubsub_v1.subscriber.message.Message) -> None:
    print(f"Received {message}.")
    message.ack()
    

In [22]:
streaming_pull_future = subscriber.subscribe(subscription_path, callback=callback)
print(f"Listening for messages on {subscription_path}..\n")

Listening for messages on projects/crypto-busting-374123/subscriptions/bda-coinbase-topic-sub..



In [23]:
with subscriber:
    try:
        # When `timeout` is not set, result() will block indefinitely,
        # unless an exception is encountered first.
        streaming_pull_future.result(timeout=timeout)
    except TimeoutError:
        streaming_pull_future.cancel()  # Trigger the shutdown.
        streaming_pull_future.result()  # Block until the shutdown is complete.

Received Message {
  data: b'{"side":"","price":"1559.29","product_id":"ETH-USD...'
  ordering_key: ''
  attributes: {}
}.
Received Message {
  data: b'{"side":"","price":"21014.69","product_id":"BTC-US...'
  ordering_key: ''
  attributes: {}
}.
Received Message {
  data: b'{"side":"","price":"0.3567","product_id":"ADA-USD"...'
  ordering_key: ''
  attributes: {}
}.
Received Message {
  data: b'{"side":"","price":"0.0886","product_id":"DOGE-USD...'
  ordering_key: ''
  attributes: {}
}.
Received Message {
  data: b'{"side":"","price":"20.21","product_id":"SOL-USD",...'
  ordering_key: ''
  attributes: {}
}.
